In [43]:
import os
import sys
import pandas as pd
import logging
from google.cloud import bigquery
from hashlib import md5
from typing import List


# **** SETUP ****

# change to match your filesystem
DATA_DIR = "./data/"
DEFAULT_SOURCE_FILE = os.path.join(DATA_DIR, "airport_source.csv")
# change to match your gcloud project 
PROJECT_NAME = "deb-01-372116"
DATASET_NAME = "airline_ticket_processor"


# **** TABLE SCHEMAS ****

TABLE_METADATA = {
    'f_tickets': {
        'table_name': 'f_tickets',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('eticket_num', 'string', mode='REQUIRED'),
            bigquery.SchemaField('airline_iata', 'string', mode='NULLABLE'),
            bigquery.SchemaField('confirmation', 'string', mode='NULLABLE'),
            bigquery.SchemaField('ticket_date', 'string', mode='NULLABLE'),
            bigquery.SchemaField('org_iata_dest_iata', 'string', mode='NULLABLE'),
            bigquery.SchemaField('price', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('seat', 'string', mode='NULLABLE'),
            bigquery.SchemaField('status', 'string', mode='NULLABLE'),
            bigquery.SchemaField('UUID_SK', 'STRING', mode='NULLABLE'),
        ],
    },
}


# **** SETUP LOGGING ****
# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level


# **** BIGQUERY CLIENT ****
logger.debug(f"Creating bigquery client")
client = bigquery.Client()

logger.info(f"Setup Completed")

DEBUG:root:Creating bigquery client
INFO:root:Setup Completed


In [44]:
#load airlines data
# receipts data file name
filename = DEFAULT_SOURCE_FILE
logger.debug(f"attempting to process: {filename}")

# *** always perform checks first ***
# check if the file exists
assert os.path.exists(filename), f"Data file does not exists: '{filename}'"
# check if the file contains any data. Header row alone is 78 bytes. size > 78
assert os.path.getsize(filename) > 78, f"Data file size incorrect; does not seem to contain data: '{filename}'"

# load into dataframe
df = pd.read_csv(filename, header=0)
logger.info(f"loaded {len(df.index)} rows from: {filename}")

# assign & remember receipts dataframe
source_df = df
display(source_df.head(n=10))

DEBUG:root:attempting to process: ./data/airport_source.csv
INFO:root:loaded 4096 rows from: ./data/airport_source.csv


,Unnamed: 0,airline_name,airline_iata,airline_icao,callsign,airline_country,origin_airport_name,origin_airport_city,origin_airport_country,origin_airport_iata,...,dest_long,dest_altitude,dest_timezone,eticket_num,confirmation,ticket_date,price,seat,status,org_iata_dest_iata
0,0,China Eastern Airlines,MU,CES,CHINA EASTERN,China,Montreal / Pierre Elliott Trudeau Internationa...,Montreal,Canada,YUL,...,-87.75,620.0,America/Chicago,498-938211-0795,ZVFDC4,2022-03-23,723.42,31I,active,YUL-MDW
1,1,Hawaiian Airlines,HA,HAL,HAWAIIAN,United States,Longdongbao Airport,Guiyang,China,KWE,...,121.02,75.0,Asia/Manila,482-850738-6048,IL5GUI,2022-03-23,765.18,29B,active,KWE-MNL
2,2,Wizz Air,W6,WZZ,WIZZ AIR,Hungary,Licenciado Gustavo Díaz Ordaz International Ai...,Puerto Vallarta,Mexico,PVR,...,1.37,24.0,Europe/Madrid,275-207321-8092,CYEFBC,2022-03-21,753.89,26I,active,PVR-IBZ
3,3,AirAsia,AK,AXM,ASIAN EXPRESS,Malaysia,El Tepual Airport,Puerto Montt,Chile,PMC,...,18.47,489.0,Europe/Warsaw,246-793315-3102,ZNGPC2,2022-03-22,793.89,15A,active,PMC-GDN
4,4,Xiamen Airlines,MF,CXA,XIAMEN AIR,China,Baltimore/Washington International Thurgood Ma...,Baltimore,United States,BWI,...,-0.19,202.0,Europe/London,091-128904-1226,MGSBD9,2022-03-24,820.25,17F,active,BWI-LGW
5,5,Air New Zealand,NZ,ANZ,NEW ZEALAND,New Zealand,Platov International Airport,Rostov,Russia,ROV,...,-84.67,896.0,America/New_York,115-196069-8963,XFYQC0,2022-03-23,892.69,18C,active,ROV-CVG
6,6,Jeju Air,7C,JJA,JEJU AIR,Republic of Korea,Winnipeg / James Armstrong Richardson Internat...,Winnipeg,Canada,YWG,...,127.65,12.0,Asia/Tokyo,396-673460-1326,N5UOOZ,2022-03-23,889.53,3C,active,YWG-OKA
7,7,American Airlines,AA,AAL,AMERICAN,United States,Ontario International Airport,Ontario,United States,ONT,...,96.13,109.0,Asia/Rangoon,380-894599-8109,PAA19Y,2022-03-22,706.78,7D,active,ONT-RGN
8,8,Juneyao Airlines,HO,DKH,JUNEYAO AIRLINES,China,Luis Munoz Marin International Airport,San Juan,Puerto Rico,SJU,...,60.80,764.0,Asia/Yekaterinburg,614-960971-2686,EF4BHJ,2022-03-23,486.40,24J,active,SJU-SVX
9,9,Royal Air Maroc,AT,RAM,ROYALAIR MAROC,Morocco,Edmonton International Airport,Edmonton,Canada,YEG,...,112.63,2575.0,Asia/Shanghai,481-321233-0702,FVM9EE,2022-03-23,855.93,16A,active,YEG-TYN


In [45]:
#  SCD2_d_passengers table query
query = f"""
SELECT
  UUID_SK,
  email
FROM 
  `{PROJECT_NAME}.{DATASET_NAME}.SCD2_d_passengers`
"""
# query bigquery table into dataframe
pdf = client.query(query).to_dataframe()
# index by product_name for the join
pdf = pdf.set_index(keys='email')
logger.info(f"query {len(pdf.index)} rows from SCD2_d_passengers table")


# drop product_id and customer_id columns if they already exist
#   this line will allow us to re-run this cell
df = df.drop(columns=['UUID_SK'], errors='ignore')

logger.debug(f"preparing for join")
# join with the main dataframe to get product_id and customer_id
df = df.join(pdf, on='email', how='inner')
logger.info(f"looked up UUID_SK")

# sort back by receipt number
logger.debug(f"sorting back by receipt id to get the rows")
df = df.sort_values(by=['UUID_SK'])


logger.debug(f"dataframe:")
display(df.head(n=10))

INFO:root:query 128 rows from SCD2_d_passengers table
DEBUG:root:preparing for join
INFO:root:looked up UUID_SK
DEBUG:root:sorting back by receipt id to get the rows
DEBUG:root:dataframe:


,Unnamed: 0,airline_name,airline_iata,airline_icao,callsign,airline_country,origin_airport_name,origin_airport_city,origin_airport_country,origin_airport_iata,...,dest_altitude,dest_timezone,eticket_num,confirmation,ticket_date,price,seat,status,org_iata_dest_iata,UUID_SK
1199,1199,Air China,CA,CCA,AIR CHINA,China,Manchester Airport,Manchester,United Kingdom,MAN,...,31.0,Asia/Manila,900-882266-8274,ZOB7W2,2022-03-24,756.04,2B,active,MAN-CEB,00e3548b-f105-40b4-99c8-eb388d64bc24
2046,2046,Air Canada,AC,ACA,AIR CANADA,Canada,Capitan FAP Carlos Martinez De Pinillos Intern...,Trujillo,Peru,TRU,...,228.0,Europe/Lisbon,473-532942-5640,SNY419,2022-03-23,285.86,18D,active,TRU-OPO,00e3548b-f105-40b4-99c8-eb388d64bc24
2029,2029,Skymark Airlines,BC,SKY,SKYMARK,Japan,Bologna Guglielmo Marconi Airport,Bologna,Italy,BLQ,...,292.0,America/Asuncion,226-774614-3341,II2ZVI,2022-03-22,507.45,20G,active,BLQ-ASU,00e3548b-f105-40b4-99c8-eb388d64bc24
2029,2029,Skymark Airlines,BC,SKY,SKYMARK,Japan,Bologna Guglielmo Marconi Airport,Bologna,Italy,BLQ,...,292.0,America/Asuncion,226-774614-3341,II2ZVI,2022-03-22,507.45,20G,active,BLQ-ASU,00e3548b-f105-40b4-99c8-eb388d64bc24
2029,2029,Skymark Airlines,BC,SKY,SKYMARK,Japan,Bologna Guglielmo Marconi Airport,Bologna,Italy,BLQ,...,292.0,America/Asuncion,226-774614-3341,II2ZVI,2022-03-22,507.45,20G,active,BLQ-ASU,00e3548b-f105-40b4-99c8-eb388d64bc24
2029,2029,Skymark Airlines,BC,SKY,SKYMARK,Japan,Bologna Guglielmo Marconi Airport,Bologna,Italy,BLQ,...,292.0,America/Asuncion,226-774614-3341,II2ZVI,2022-03-22,507.45,20G,active,BLQ-ASU,00e3548b-f105-40b4-99c8-eb388d64bc24
1996,1996,Hawaiian Airlines,HA,HAL,HAWAIIAN,United States,NaN,NaN,NaN,NaN,...,489.0,America/Tijuana,368-016150-5337,S4JZVJ,2022-03-23,755.99,27D,active,nan-TIJ,00e3548b-f105-40b4-99c8-eb388d64bc24
2046,2046,Air Canada,AC,ACA,AIR CANADA,Canada,Capitan FAP Carlos Martinez De Pinillos Intern...,Trujillo,Peru,TRU,...,228.0,Europe/Lisbon,473-532942-5640,SNY419,2022-03-23,285.86,18D,active,TRU-OPO,00e3548b-f105-40b4-99c8-eb388d64bc24
1996,1996,Hawaiian Airlines,HA,HAL,HAWAIIAN,United States,NaN,NaN,NaN,NaN,...,489.0,America/Tijuana,368-016150-5337,S4JZVJ,2022-03-23,755.99,27D,active,nan-TIJ,00e3548b-f105-40b4-99c8-eb388d64bc24
1996,1996,Hawaiian Airlines,HA,HAL,HAWAIIAN,United States,NaN,NaN,NaN,NaN,...,489.0,America/Tijuana,368-016150-5337,S4JZVJ,2022-03-23,755.99,27D,active,nan-TIJ,00e3548b-f105-40b4-99c8-eb388d64bc24


In [46]:
#define the load_table function
def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: List[bigquery.SchemaField], 
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    """load dataframe into bigquery table

    Args:
        df (pd.DataFrame): dataframe to load
        client (bigquery.Client): bigquery client
        table_name (str): full table name including project and dataset id
        schema (List[bigquery.SchemaField]): table schema with data types
        create_disposition (str, optional): create table disposition. Defaults to 'CREATE_IF_NEEDED'.
        write_disposition (str, optional): overwrite table disposition. Defaults to 'WRITE_TRUNCATE'.
    """
    # *** run some checks ***
    # test table name to be full table name including project and dataset name. It must contain to dots
    assert len(table_name.split('.')) == 3, f"Table name must be a full bigquery table name including project and dataset id: '{table_name}'"
    # setup bigquery load job:
    #  create table if needed, replace rows, define the table schema
    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    logger.info(f"loading table: '{table_name}'")
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result()        # wait for the job to finish
    # get the resulting table
    table = client.get_table(table_name)
    logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")

In [47]:
##----------------load d_airlines---------------------
# get unique receipts (not line items)
f_tickets = df[['eticket_num','airline_iata', 'confirmation', 'ticket_date','org_iata_dest_iata', 'price', 'seat','status','UUID_SK']].drop_duplicates(keep='first')
logger.info(f"Preparing {len(f_tickets.index)} unique receipts to load to bigquery")
f_tickets = f_tickets.astype({"UUID_SK": str})

display(f_tickets.head(n=10))

# load to bigquery
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['f_tickets']['table_name']}"
schema = TABLE_METADATA['f_tickets']['schema']
load_table(f_tickets, client, table_name, schema)

INFO:root:Preparing 4096 unique receipts to load to bigquery


,eticket_num,airline_iata,confirmation,ticket_date,org_iata_dest_iata,price,seat,status,UUID_SK
1199,900-882266-8274,CA,ZOB7W2,2022-03-24,MAN-CEB,756.04,2B,active,00e3548b-f105-40b4-99c8-eb388d64bc24
2046,473-532942-5640,AC,SNY419,2022-03-23,TRU-OPO,285.86,18D,active,00e3548b-f105-40b4-99c8-eb388d64bc24
2029,226-774614-3341,BC,II2ZVI,2022-03-22,BLQ-ASU,507.45,20G,active,00e3548b-f105-40b4-99c8-eb388d64bc24
1996,368-016150-5337,HA,S4JZVJ,2022-03-23,nan-TIJ,755.99,27D,active,00e3548b-f105-40b4-99c8-eb388d64bc24
1965,842-360341-3828,MH,AW4NX7,2022-03-21,FCO-NAT,242.10,30F,active,00e3548b-f105-40b4-99c8-eb388d64bc24
1960,704-114952-8642,HR,V5XKIQ,2022-03-23,BOD-ZAG,409.65,1F,active,00e3548b-f105-40b4-99c8-eb388d64bc24
2077,232-900389-1685,U6,8BXEAZ,2022-03-21,SLZ-CGO,253.43,9E,active,00e3548b-f105-40b4-99c8-eb388d64bc24
2096,036-291604-5866,SJ,EH06TD,2022-03-23,ZAG-GYN,887.90,31A,active,00e3548b-f105-40b4-99c8-eb388d64bc24
2087,285-050142-3653,WN,22L30Q,2022-03-23,ANC-CSX,610.51,32A,active,00e3548b-f105-40b4-99c8-eb388d64bc24
2085,142-130089-0576,NZ,EL3FZE,2022-03-22,GRR-KZN,908.91,4H,active,00e3548b-f105-40b4-99c8-eb388d64bc24


INFO:root:loading table: 'deb-01-372116.airline_ticket_processor.f_tickets'
INFO:root:loaded 4096 rows into deb-01-372116:airline_ticket_processor.f_tickets
